<a href="https://colab.research.google.com/github/Phimphika113/sobeam_year4/blob/main/nurse_w_weekly_workforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyomo ## ติดตั้ง Pyomo
!apt-get install -y glpk-utils  ##Solver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
G

In [2]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Define model parameters

In [54]:
# Define days
days = range(1, 31) # เซตวันแต่ละเดือน 31 วัน

# Enter shifts of each day
shifts_s = ['morning', 'evening', 'night']  # เซตผลัด
days_shifts_k = {day: shifts_s for day in days}  #ดัชนีผลัด

# Enter nurse ids (name, number, ...)
nurse_N = range(1, 18)  # เซตพยาบาล 17 คน

# Age  of nurse
age_nurse_R = [0] * 17  # เริ่มต้นด้วย 0
age_nurse_R[0] = 2 # ตั้งค่าตัวที่ 1 และ 2 เป็น 2
age_nurse_R[1] = 2
for i in range(2, 17): # ตั้งค่าตัวที่ 3 ถึง 17 เป็น 1
    age_nurse_R[i] = 1

# Age  of nurse
#import numpy as np #เซตอายุของพยาบาล
#age_nurse_R = np.ones(17) #สร้างอาเรย์ยาว 17 และค่าเริ่มต้นเป็น 1
#age_nurse_R[0] = 2 #ค่าตัวที่ 1 และ 2 เป็น 2
#age_nurse_R[1] = 2

# ข้อมูลการมีโรคประจำตัว (มีแค่พยบ 1 คน แล้วตัดออกแล้ว)
#Disease_nurse = np.zeros(17) #สร้างอาเรย์ยาว 17 และค่าเริ่มต้นเป็น 0
#Disease_nurse[0] = 1 #ค่าตัวที่ 1 และ 2 เป็น 2
#Disease_nurse[1] = 1

#กำหนดอายุงานพยาบาล
# เซตอายุงานพยาบาล
work_experience_L = {1: 18,
                              2: 18,
                              3: 15,
                              4: 14,
                              5: 14,
                              6: 12,
                              7: 15,
                              8: 10,
                              9: 9,
                              10: 12,
                              11: 10,
                              12: 9,
                              13: 8,
                              14: 6,
                              15: 1,
                              16: 1,
                              17: 1}

In [42]:
# Create a DataFrame to store nurse information
import pandas as pd
df_nurse = pd.DataFrame ({'Nurse_ID': nurse_N,
                          'Age': age_nurse_R,
                          'Work_Experience': [work_experience_L[i+1] for i in range(17)]})

# Display the DataFrame
print(df_nurse.to_string(index=False))

 Nurse_ID  Age  Work_Experience
        1    2               18
        2    2               18
        3    1               15
        4    1               14
        5    1               14
        6    1               12
        7    1               15
        8    1               10
        9    1                9
       10    1               12
       11    1               10
       12    1                9
       13    1                8
       14    1                6
       15    1                1
       16    1                1
       17    1                1


#Initialize model and variables


In [55]:
# Initialize model
model = ConcreteModel()

# binary variables แสดงการได้รับตารางงานของพยาบาล (Normaltime)
model.nurse_work_X = Var(((i, j, k) for i in nurse_N for j in days for k in days_shifts_k[j]),
                  within=Binary, initialize=0)
# model.nurse_work_Z = Var(((nurse, day, shift) for nurse in nurse_N for day in days for shift in days_shifts_k[day]),
#                 within=Binary, initialize=0)  # ตัวแปรการทำงานในผลัดล่วงเวลา


#Define the objective function

In [ ]:
# Define an objective function with model as input, to pass later
def obj_fun(NT):
  model.NT = sum(model.nurse_work_X[i, j, k] for i in nurse_N for j in days for k in days_shifts_k[j])  # ผลัดการทำงานทั้งหมดของพยาบาล i ในรอบ 1 เดือน
  alpha = model.NT / len(nurse_N)  # avg of NT, use model.NT here
  differ_NT =   model.NT- alpha
  return sum(((m.NT[nurse_N] for i in nurse_N) - alpha)*((m.NT[nurse_N] for i in nurse_N) - alpha))

# add objective function to the model. rule (pass function) or expr (pass expression directly)
model.obj = Objective(rule=obj_fun, sense=minimize)

In [71]:
# Define an objective function with model as input, to pass later
def obj_fun(NT):
  model.NT = sum(model.nurse_work_X[i, j, k] for i in nurse_N for j in days for k in days_shifts_k[j])  # ผลัดการทำงานทั้งหมดของพยาบาล i ในรอบ 1 เดือน
  alpha = model.NT / len(nurse_N)  # avg of NT, use model.NT here
  differ_NT = model.NT- alpha
  return sum(differ_NT**2 for _ in nurse_N) # Square the difference and sum over all nurses

# add objective function to the model. rule (pass function) or expr (pass expression directly)
model.obj = Objective(rule=obj_fun, sense=minimize)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


#Model constraints